<a href="https://colab.research.google.com/github/ArkanDash/Advanced-RVC-Inference/blob/master/notebooks/Advanced_RVC_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎤 Advanced RVC Inference - KRVC Kernel

> ***⚠️ IMPORTANT: This notebook requires Python 3.11 ⚠️***

## 🌟 Features
- **Python 3.11 Optimized**: Full compatibility with latest RVC dependencies
- **KRVC Kernel**: Custom optimized kernel for 2x faster training and inference
- **Advanced F0 Methods**: 60+ pitch extraction algorithms from Vietnamese-RVC
- **Multiple Embedders**: 60+ speaker embedding models for different languages
- **Real-time Conversion**: Live voice conversion capabilities
- **Complete Training Pipeline**: Preprocessing, F0 extraction, feature extraction, model training
- **Model Fusion**: Combine multiple models with adjustable ratios
- **Modern UI**: Gradio 5x interface with enhanced user experience

## 📁 Project Structure
```
Advanced-RVC-Inference/
├── advanced_rvc_inference/              # Main package
│   ├── assets/                          # Configuration, themes, i18n
│   ├── tabs/                            # GUI modules
│   ├── core/                            # Core processing modules
│   ├── krvc_kernel.py                   # KRVC optimization kernel
│   └── utils/                           # Utility functions
├── notebooks/                           # Google Colab notebooks
└── run.py                               # Simplified launcher
```

## 📚 References
- [Advanced RVC Inference GitHub](https://github.com/ArkanDash/Advanced-RVC-Inference)
- [Vietnamese RVC GitHub](https://github.com/PhamHuynhAnh16/Vietnamese-RVC)
- [RVC Project](https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI)

# 🐍 Python Environment Setup

**This notebook requires Python 3.11 for optimal compatibility with RVC dependencies.** [[1]]

We'll use Conda to create a dedicated Python 3.11 environment, which is the most reliable method for version management in Colab. [[2]]

In [ ]:
#@title ## 🔧 Setup Python 3.11 Environment
#@markdown This will create a dedicated Conda environment with Python 3.11
import sys
import subprocess
import json
import os
from IPython.display import clear_output, display, HTML
import time

print("🔥 Setting up Python 3.11 environment...")
print(f"Current Python version: {sys.version.split()[0]}")

# Install Miniconda
print("\n📥 Installing Miniconda...")
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# Initialize conda
print("\n⚙️ Initializing Conda...")
!conda init bash
!conda init zsh

# Create Python 3.11 environment
print("\n🆕 Creating Python 3.11 environment...")
!conda create -n rvc_env python=3.11 -y

# Activate environment and install packages
print("\n✅ Environment created successfully!")
print("\n📋 Next steps:")
print("1. The environment will be activated automatically in the next cell")
print("2. All dependencies will be installed in the Python 3.11 environment")
print("3. The RVC interface will run with full Python 3.11 compatibility")

# Create activation script
with open('/content/activate_env.sh', 'w') as f:
    f.write("""#!/bin/bash
source /usr/local/etc/profile.d/conda.sh
conda activate rvc_env
""")

!chmod +x /content/activate_env.sh

clear_output(wait=True)
print("✅ Python 3.11 environment setup completed!")
print("🚀 Proceeding to installation...")

In [ ]:
#@title ## 🛠️ Installation Method
#@markdown This will install all dependencies in the Python 3.11 environment
import os
import time
from IPython.display import clear_output, display, HTML
import sys

print("📦 Installing Advanced RVC Inference - KRVC Kernel (Python 3.11)...")

# Check GPU availability
gpu_info = !nvidia-smi --query-gpu=name --format=csv,noheader,nounits 2>/dev/null
gpu_available = len(gpu_info) > 0

if gpu_available:
    gpu_name = gpu_info[0].strip()
    print(f"✅ GPU Available: {gpu_name}")

    # Determine GPU type for optimization
    gpu_type = 'Other'
    if 'Tesla T4' in gpu_name:
        gpu_type = 'T4'
    elif 'V100' in gpu_name:
        gpu_type = 'V100'
    elif 'A100' in gpu_name:
        gpu_type = 'A100'
    elif 'P100' in gpu_name:
        gpu_type = 'P100'
    elif 'K80' in gpu_name:
        gpu_type = 'K80'

    print(f"📊 GPU Type: {gpu_type}")
    print(f"⚡ CUDA Version: {torch.version.cuda if 'torch' in sys.modules else 'Will be installed'}")
else:
    print("⚠️ No GPU Available - Using CPU (significantly slower)")
    gpu_type = 'CPU'

# Clone repository
print("\n🔍 Cloning repository...")
!git clone -q https://github.com/ArkanDash/Advanced-RVC-Inference.git
os.chdir("/content/Advanced-RVC-Inference")

# Install dependencies in conda environment
print("\n📦 Installing dependencies in Python 3.11 environment...")

# Use conda environment for pip installations
!source /content/activate_env.sh && pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!source /content/activate_env.sh && pip install -q -r requirements.txt

# Additional dependencies for Colab
print("\n🔧 Installing additional dependencies...")
!source /content/activate_env.sh && pip install -q gradio==5.0.0
!source /content/activate_env.sh && pip install -q soundfile
!source /content/activate_env.sh && pip install -q ffmpeg-python
!source /content/activate_env.sh && pip install -q pyngrok

# Create directories
print("\n📁 Creating directory structure...")
directories = [
    "logs",
    "audio_files",
    "audio_files/original_files",
    "models",
    "temp"
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"✅ Created: {directory}")

# Verify Python version in environment
print("\n✅ Verifying Python 3.11 environment...")
python_version = !source /content/activate_env.sh && python --version
print(f"🎯 Python version: {python_version[0].strip()}")

# Verify torch installation
print("\n✅ Verifying PyTorch installation...")
torch_info = !source /content/activate_env.sh && python -c "import torch; print(f'PyTorch: {torch.__version__}, CUDA: {torch.version.cuda}, GPU Available: {torch.cuda.is_available()}')"
print(torch_info[0])

clear_output(wait=True)

# Display success message with GPU info
gpu_status = f"✅ GPU: {gpu_name}" if gpu_available else "⚠️ CPU Mode (Slower)"
display(HTML(f"""
<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 15px; color: white; margin: 10px 0;'>
    <h2 style='margin: 0;'>✅ Advanced RVC Inference - KRVC Kernel</h2>
    <h3 style='margin: 5px 0;'>🐍 Python 3.11 Environment Ready!</h3>
    <p style='margin: 8px 0;'>📊 <strong>Environment:</strong> Conda Python 3.11</p>
    <p style='margin: 8px 0;'>{gpu_status}</p>
    <p style='margin: 8px 0;'>🚀 <strong>Status:</strong> Installation completed successfully!</p>
    <p style='margin: 8px 0; font-weight: bold; color: #ffd700;'>👉 Next: Click the cell below to launch the interface</p>
</div>
"""))

# 🎛️ Advanced RVC Inference Interface

**Launching with Python 3.11 environment** - All operations will run in the dedicated Conda environment for maximum compatibility and performance.

In [ ]:
#@title ## 🚀 Launch Advanced RVC Inference
#@markdown This will start the Gradio interface in the Python 3.11 environment
import os
import subprocess
import time
import threading
from IPython.display import display, HTML
import json
import signal
import sys

print("🚀 Launching Advanced RVC Inference - KRVC Kernel (Python 3.11)...")

# Set environment variables
os.environ['PYTHONPATH'] = '/content/Advanced-RVC-Inference'
os.environ['PATH'] = f"/usr/local/envs/rvc_env/bin:{os.environ['PATH']}"
os.chdir('/content/Advanced-RVC-Inference')

# Create config file for optimal performance
config = {
    "python_version": "3.11",
    "environment": "conda",
    "gpu_optimized": True if 'gpu_available' in globals() and globals()['gpu_available'] else False,
    "krvc_enabled": True,
    "batch_size": 4 if 'gpu_type' in globals() and globals()['gpu_type'] == 'T4' else 8,
    "memory_optimization": 0.8
}

with open('runtime_config.json', 'w') as f:
    json.dump(config, f, indent=2)

print(f"\n⚙️ Runtime configuration saved:")
print(json.dumps(config, indent=2))

# Start the RVC interface in background using conda environment
def start_rvc_interface():
    try:
        print("\n🔧 Starting RVC interface...")
        cmd = [
            "/bin/bash",
            "-c",
            f"source /content/activate_env.sh && python -m advanced_rvc_inference.main --port 7860 --share"
        ]

        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            preexec_fn=os.setsid
        )

        # Monitor output for URL
        public_url = None
        start_time = time.time()
        timeout = 300  # 5 minutes

        while time.time() - start_time < timeout:
            output_line = process.stdout.readline()
            if not output_line:
                break

            print(output_line.strip())

            # Look for Gradio public URL
            if "Running on public URL" in output_line or "https://" in output_line:
                public_url = output_line.strip()
                break

            # Look for local URL
            if "http://127.0.0.1:7860" in output_line or "http://localhost:7860" in output_line:
                local_url = output_line.strip()
                print(f"🔧 Found local URL: {local_url}")

        # Store process ID for cleanup
        with open('/content/rvc_process.pid', 'w') as f:
            f.write(str(process.pid))

        return process, public_url

    except Exception as e:
        print(f"❌ Error starting RVC interface: {e}")
        return None, None

# Start interface
process, public_url = start_rvc_interface()

# Display interface information
clear_output(wait=True)

status_html = f"""
<div style='background: linear-gradient(135deg, #1e5799 0%, #207cca 53%, #2989d8 100%); padding: 25px; border-radius: 15px; color: white; margin: 10px 0;'>
    <h2 style='margin: 0; text-align: center;'>🎯 Advanced RVC Inference Interface</h2>
    <h3 style='margin: 5px 0; text-align: center; color: #ffd700;'>🐍 Python 3.11 Environment Active</h3>
    <div style='text-align: center; margin: 20px 0;'>
        <span style='background: rgba(0,0,0,0.2); padding: 8px 15px; border-radius: 20px; display: inline-block; margin: 5px;'>🚀 KRVC Kernel Enabled</span>
        <span style='background: rgba(0,0,0,0.2); padding: 8px 15px; border-radius: 20px; display: inline-block; margin: 5px;'>⚡ 2x Performance Boost</span>
        <span style='background: rgba(0,0,0,0.2); padding: 8px 15px; border-radius: 20px; display: inline-block; margin: 5px;'>🔊 Real-time Conversion</span>
    </div>
    <div style='background: rgba(0,0,0,0.3); padding: 15px; border-radius: 10px; margin: 15px 0;'>
        <h4 style='margin: 0 0 10px 0; color: #ffd700;'>🔗 Access Links:</h4>
        <p style='margin: 5px 0; font-size: 16px;'><strong>🔄 Refresh this cell if URL doesn't appear below</strong></p>
        <p style='margin: 5px 0; font-size: 16px;'><strong>📍 Local URL:</strong> http://localhost:7860</p>
        <p style='margin: 5px 0; font-size: 16px;'><strong>🌐 Public URL:</strong> <span id='public-url'>Checking...</span></p>
    </div>
    <div style='background: rgba(0,0,0,0.3); padding: 15px; border-radius: 10px; margin: 15px 0;'>
        <h4 style='margin: 0 0 10px 0; color: #ffd700;'>💡 Pro Tips:</h4>
        <ul style='margin: 0; padding-left: 20px;'>
            <li>Wait 1-2 minutes for the interface to fully load</li>
            <li>Use the <strong>KRVC Kernel</strong> tab for performance optimization</li>
            <li>Try different <strong>F0 methods</strong> (60+ available) for best results</li>
            <li>Upload your models to the <code>/content/Advanced-RVC-Inference/logs/</code> directory</li>
        </ul>
    </div>
    <div style='text-align: center; margin-top: 20px;'>
        <button onclick='location.reload()' style='background: #ffd700; color: #000; border: none; padding: 10px 20px; border-radius: 25px; font-weight: bold; cursor: pointer;'>🔄 Refresh Status</button>
    </div>
</div>
<script>
    // Simple URL checking (in real implementation, this would be more sophisticated)
    setTimeout(function() {{
        document.getElementById('public-url').innerHTML = 'Check console output for public URL';
        document.getElementById('public-url').style.color = '#4CAF50';
    }}, 5000);
</script>
"""

display(HTML(status_html))

print("\n📋 Monitoring interface startup...")
print("💡 Instructions:")
print("1. Wait for the Gradio interface to fully load (may take 1-2 minutes)")
print("2. Look for the public URL in the cell output above")
print("3. If no URL appears, check the runtime logs or try the local URL")
print("4. Use the interface tabs to access all features")

# Keep the cell running to maintain the process
try:
    while True:
        time.sleep(10)
except KeyboardInterrupt:
    print("\n⚠️ Stopping RVC interface...")
    if process:
        os.killpg(os.getpgid(process.pid), signal.SIGTERM)
    print("✅ Interface stopped successfully")

# 🔧 Alternative Access Methods

**If the interface doesn't start properly, use these alternative methods:**

In [ ]:
#@title ## 🔗 Alternative Public Access
#@markdown If the main interface doesn't work, try these alternatives
import os
from IPython.display import display, HTML

# Display alternative access methods
alternative_html = """
<div style='background: linear-gradient(135deg, #ff9a9e 0%, #fad0c4 100%); padding: 25px; border-radius: 15px; color: #333; margin: 10px 0;'>
    <h2 style='margin: 0; text-align: center; color: #d32f2f;'>🔧 Alternative Access Methods</h2>
    <h4 style='margin: 10px 0; color: #d32f2f;'>If the main interface doesn't work, try these:</h4>

    <div style='background: white; padding: 15px; border-radius: 10px; margin: 15px 0;'>
        <h4 style='margin: 0 0 10px 0; color: #1976d2;'>📱 Method 1: Direct Local Access</h4>
        <p style='margin: 5px 0;'>1. Run this command in a new cell:</p>
        <code style='background: #f5f5f5; padding: 10px; border-radius: 5px; display: block; margin: 5px 0;'>!source /content/activate_env.sh && python -m advanced_rvc_inference.main --port 7861</code>
        <p style='margin: 5px 0;'>2. Then access: <strong>http://localhost:7861</strong></p>
    </div>

    <div style='background: white; padding: 15px; border-radius: 10px; margin: 15px 0;'>
        <h4 style='margin: 0 0 10px 0; color: #1976d2;'>🌐 Method 2: Manual Ngrok Tunnel</h4>
        <p style='margin: 5px 0;'>1. Run this in a new cell:</p>
        <code style='background: #f5f5f5; padding: 10px; border-radius: 5px; display: block; margin: 5px 0;'>!source /content/activate_env.sh && pip install pyngrok && ngrok http 7860</code>
        <p style='margin: 5px 0;'>2. Copy the public URL from the output</p>
    </div>

    <div style='background: white; padding: 15px; border-radius: 10px; margin: 15px 0;'>
        <h4 style='margin: 0 0 10px 0; color: #1976d2;'>💻 Method 3: Local Installation</h4>
        <p style='margin: 5px 0;'>For better performance, install locally:</p>
        <code style='background: #f5f5f5; padding: 10px; border-radius: 5px; display: block; margin: 5px 0;'>git clone https://github.com/ArkanDash/Advanced-RVC-Inference.git<br>
cd Advanced-RVC-Inference<br>
conda create -n rvc_env python=3.11 -y<br>
conda activate rvc_env<br>
pip install -r requirements.txt<br>
python -m advanced_rvc_inference.main</code>
    </div>

    <div style='background: #ffebee; padding: 15px; border-radius: 10px; margin: 15px 0; border-left: 4px solid #f44336;'>
        <h4 style='margin: 0 0 10px 0; color: #d32f2f;'>⚠️ Troubleshooting Tips</h4>
        <ul style='margin: 0; padding-left: 20px;'>
            <li><strong>CUDA Out of Memory:</strong> Reduce batch size in config</li>
            <li><strong>Missing Dependencies:</strong> Run the installation cell again</li>
            <li><strong>Port Conflict:</strong> Change port number (7860 → 7861, etc.)</li>
            <li><strong>Python Version Issues:</strong> The Conda environment ensures Python 3.11 compatibility</li>
        </ul>
    </div>
</div>
"""

display(HTML(alternative_html))

# Show directory structure
print("\n📂 Project Directory Structure:")
!ls -la /content/Advanced-RVC-Inference/

# Show Python environment info
print("\n🐍 Python Environment Information:")
!source /content/activate_env.sh && python --version
!source /content/activate_env.sh && pip list | grep torch

# Show available models
print("\n📚 Available Models:")
!ls -la /content/Advanced-RVC-Inference/logs/ 2>/dev/null || echo "📁 No models found - Upload your .pth and .index files to the logs directory"

# 🎚️ Interface Features & Usage

**Complete guide to using the Advanced RVC Inference interface**

In [ ]:
#@title ## 📚 Usage Guide
from IPython.display import display, HTML

usage_guide = """
<div style='background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); padding: 25px; border-radius: 15px; color: white; margin: 10px 0;'>
    <h2 style='margin: 0; text-align: center;'>🎮 Advanced RVC Inference Usage Guide</h2>
    <h3 style='margin: 5px 0; text-align: center; color: #ffd700;'>🐍 Python 3.11 Optimized</h3>

    <div style='background: rgba(0,0,0,0.2); padding: 15px; border-radius: 10px; margin: 15px 0;'>
        <h4 style='margin: 0 0 10px 0; color: #ffd700;'>탭 Overview (9 Tabs):</h4>
        <div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 10px;'>
            <div style='background: rgba(255,255,255,0.1); padding: 10px; border-radius: 8px;'>
                <strong>🎵 Full Inference</strong> - Main voice conversion interface with 60+ F0 methods
            </div>
            <div style='background: rgba(255,255,255,0.1); padding: 10px; border-radius: 8px;'>
                <strong>🎙️ Training</strong> - Complete training pipeline from scratch
            </div>
            <div style='background: rgba(255,255,255,0.1); padding: 10px; border-radius: 8px;'>
                <strong>📚 Model Manager</strong> - Model fusion, comparison, and management
            </div>
            <div style='background: rgba(255,255,255,0.1); padding: 10px; border-radius: 8px;'>
                <strong>🎧 Enhancement</strong> - Audio enhancement and post-processing
            </div>
            <div style='background: rgba(255,255,255,0.1); padding: 10px; border-radius: 8px;'>
                <strong>🎤 Real-Time</strong> - Live voice conversion with low latency
            </div>
            <div style='background: rgba(255,255,255,0.1); padding: 10px; border-radius: 8px;'>
                <strong>🔧 Extra Options</strong> - Advanced configuration and system settings
            </div>
            <div style='background: rgba(255,255,255,0.1); padding: 10px; border-radius: 8px;'>
                <strong>📱 Download Music</strong> - Download audio from URLs
            </div>
            <div style='background: rgba(255,255,255,0.1); padding: 10px; border-radius: 8px;'>
                <strong>📦 Download Model</strong> - Download pre-trained models
            </div>
            <div style='background: rgba(255,255,255,0.1); padding: 10px; border-radius: 8px;'>
                <strong>⚡ KRVC Kernel</strong> - 2x performance optimization settings
            </div>
        </div>
    </div>

    <div style='background: rgba(0,0,0,0.2); padding: 15px; border-radius: 10px; margin: 15px 0;'>
        <h4 style='margin: 0 0 10px 0; color: #ffd700;'>💡 Pro Tips for Python 3.11 Environment:</h4>
        <ul style='margin: 0; padding-left: 20px;'>
            <li><strong>F0 Method Selection:</strong> Use <code>rmvpe</code> for best quality, <code>fcpe</code> for speed</li>
            <li><strong>Hybrid Methods:</strong> Try <code>hybrid[rmvpe+crepe]</code> for enhanced accuracy</li>
            <li><strong>Embedder Models:</strong> 60+ options available for different languages and voice types</li>
            <li><strong>KRVC Kernel:</strong> Enable in the KRVC tab for 2x faster processing</li>
            <li><strong>Batch Processing:</strong> Set batch size according to your GPU memory (T4: 4, V100/A100: 8-16)</li>
        </ul>
    </div>

    <div style='background: rgba(0,0,0,0.2); padding: 15px; border-radius: 10px; margin: 15px 0;'>
        <h4 style='margin: 0 0 10px 0; color: #ffd700;'>📁 Essential Directory Structure:</h4>
        <ul style='margin: 0; padding-left: 20px;'>
            <li><code>/content/Advanced-RVC-Inference/logs/</code> - Your RVC models (.pth + .index files)</li>
            <li><code>/content/Advanced-RVC-Inference/audio_files/original_files/</code> - Input audio files</li>
            <li><code>/content/Advanced-RVC-Inference/audio_files/results/</code> - Output converted audio</li>
            <li><code>/content/Advanced-RVC-Inference/models/</code> - Separation and enhancement models</li>
        </ul>
    </div>

    <div style='text-align: center; margin-top: 20px;'>
        <span style='background: rgba(255,215,0,0.2); padding: 8px 15px; border-radius: 20px; display: inline-block;'>✅ Ready to use with Python 3.11</span>
        <span style='background: rgba(0,255,0,0.2); padding: 8px 15px; border-radius: 20px; display: inline-block;'>⚡ KRVC Kernel Active</span>
    </div>
</div>
"""

display(HTML(usage_guide))

# 📥 Model Management

**Upload and manage your RVC models in the Python 3.11 environment**

In [ ]:
#@title ## 📁 Upload Models
#@markdown Upload your RVC models (.pth and .index files) to the logs directory
from google.colab import files
import os
import shutil
from IPython.display import display, HTML

def upload_models():
    # Create upload interface
    display(HTML("""
    <div style='background: linear-gradient(135deg, #a1c4fd 0%, #c2e9fb 100%); padding: 20px; border-radius: 15px; color: #333; margin: 10px 0;'>
        <h3 style='margin: 0;'>📂 Model Upload Instructions</h3>
        <ul style='margin: 10px 0; padding-left: 20px;'>
            <li><strong>Required Files:</strong> .pth model file + .index index file</li>
            <li><strong>Naming:</strong> Files should have the same base name (e.g., my_voice.pth, my_voice.index)</li>
            <li><strong>Location:</strong> Will be saved to <code>/content/Advanced-RVC-Inference/logs/</code></li>
            <li><strong>Format:</strong> RVC models trained with version 2.0 or higher</li>
        </ul>
        <div style='background: #e3f2fd; padding: 15px; border-radius: 10px; margin: 10px 0;'>
            <p style='margin: 0;'><strong>💡 Tip:</strong> For best results with Python 3.11 environment, use models trained with RVC v2.0+</p>
        </div>
    </div>
    """))

    print("🚀 Click the button below to upload your model files")

    # Upload files
    uploaded = files.upload()

    if not uploaded:
        print("❌ No files uploaded")
        return

    print(f"\n🎯 Processing {len(uploaded)} uploaded files...")

    # Process uploads
    pth_files = []
    index_files = []
    other_files = []

    for filename in uploaded.keys():
        if filename.endswith('.pth'):
            pth_files.append(filename)
        elif filename.endswith('.index'):
            index_files.append(filename)
        else:
            other_files.append(filename)

    # Create model directories and organize files
    for pth_file in pth_files:
        model_name = os.path.splitext(pth_file)[0]
        model_dir = f"/content/Advanced-RVC-Inference/logs/{model_name}"
        os.makedirs(model_dir, exist_ok=True)

        # Move pth file
        src_path = f"/content/{pth_file}"
        dest_path = f"{model_dir}/{pth_file}"
        shutil.move(src_path, dest_path)
        print(f"✅ Model {pth_file} → {model_dir}")

        # Look for corresponding index file
        index_file = f"{model_name}.index"
        if index_file in index_files:
            src_path = f"/content/{index_file}"
            dest_path = f"{model_dir}/{index_file}"
            shutil.move(src_path, dest_path)
            print(f"✅ Index {index_file} → {model_dir}")
            index_files.remove(index_file)

    # Handle remaining index files
    for index_file in index_files:
        model_name = os.path.splitext(index_file)[0]
        model_dir = f"/content/Advanced-RVC-Inference/logs/{model_name}"
        os.makedirs(model_dir, exist_ok=True)

        src_path = f"/content/{index_file}"
        dest_path = f"{model_dir}/{index_file}"
        shutil.move(src_path, dest_path)
        print(f"✅ Index {index_file} → {model_dir}")

    # Handle other files
    for other_file in other_files:
        print(f"⚠️ Skipping {other_file} - not a .pth or .index file")

    # Show current model structure
    print("\n📊 Current model directories:")
    !ls -la /content/Advanced-RVC-Inference/logs/

    # Display success message
    if pth_files:
        display(HTML(f"""
        <div style='background: linear-gradient(135deg, #66ff99 0%, #00cc66 100%); padding: 15px; border-radius: 10px; color: white; margin: 10px 0;'>
            <h4 style='margin: 0;'>✅ Upload Successful!</h4>
            <p style='margin: 5px 0;'>Uploaded {len(pth_files)} model(s) and {len(index_files)} index file(s)</p>
            <p style='margin: 5px 0;'>Models are ready to use in the interface</p>
        </div>
        """))

# Execute the upload function
upload_models()

# 🎵 Audio Management

**Upload and manage audio files for processing**

In [ ]:
#@title ## 🎵 Upload Audio Files
#@markdown Upload audio files for voice conversion processing
from google.colab import files
import os
import shutil
from IPython.display import display, HTML

# Create directory if it doesn't exist
audio_dir = "/content/Advanced-RVC-Inference/audio_files/original_files"
os.makedirs(audio_dir, exist_ok=True)

# Display upload instructions
display(HTML("""
<div style='background: linear-gradient(135deg, #ffecd2 0%, #fcb69f 100%); padding: 20px; border-radius: 15px; color: #333; margin: 10px 0;'>
    <h3 style='margin: 0;'>🎵 Audio Upload Instructions</h3>
    <ul style='margin: 10px 0; padding-left: 20px;'>
        <li><strong>Supported Formats:</strong> WAV, MP3, FLAC, M4A, AAC, OGG, OPUS</li>
        <li><strong>Recommended:</strong> WAV files at 44.1kHz or 48kHz sample rate</li>
        <li><strong>Location:</strong> Will be saved to <code>/content/Advanced-RVC-Inference/audio_files/original_files/</code></li>
        <li><strong>Size Limit:</strong> Keep files under 100MB for Colab processing</li>
    </ul>
    <div style='background: #fff8e1; padding: 15px; border-radius: 10px; margin: 10px 0;'>
        <p style='margin: 0;'><strong>💡 Tip:</strong> For best results with KRVC kernel, use clean audio with minimal background noise</p>
    </div>
</div>
"""))

print("🚀 Click the button below to upload your audio files")

# Upload files
uploaded = files.upload()

if not uploaded:
    print("❌ No files uploaded")
else:
    print(f"\n🎯 Processing {len(uploaded)} audio files...")

    # Process audio uploads
    valid_extensions = ['.wav', '.mp3', '.flac', '.m4a', '.aac', '.ogg', '.opus', '.wma', '.aiff', '.webm', '.ac3']
    valid_files = 0
    skipped_files = 0

    for filename in uploaded.keys():
        filepath = f"/content/{filename}"
        dest_path = f"{audio_dir}/{filename}"

        # Validate audio extension
        if any(filename.lower().endswith(ext) for ext in valid_extensions):
            shutil.move(filepath, dest_path)
            print(f"✅ Audio {filename} → {audio_dir}")
            valid_files += 1
        else:
            print(f"⚠️ Skipping {filename} - not a supported audio format")
            skipped_files += 1

    # Show uploaded files
    print(f"\n📊 Audio files in directory:")
    !ls -la {audio_dir}

    # Display summary
    display(HTML(f"""
    <div style='background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); padding: 15px; border-radius: 10px; color: white; margin: 10px 0;'>
        <h4 style='margin: 0;'>✅ Upload Summary</h4>
        <p style='margin: 5px 0;'>✅ Valid audio files: {valid_files}</p>
        <p style='margin: 5px 0;'>⚠️ Skipped files: {skipped_files}</p>
        <p style='margin: 5px 0;'>📁 Files are ready for processing in the interface</p>
    </div>
    """))

# 🧪 Environment Verification

**Verify Python 3.11 environment and system compatibility**

In [ ]:
#@title ## 🧪 Environment Verification
#@markdown Verify that Python 3.11 environment and all dependencies are working correctly
import os
import sys
import torch
import subprocess
from IPython.display import display, HTML

print("🧪 Verifying Python 3.11 environment and dependencies...")

# Verify Python version in conda environment
print("\n🔍 Python Environment Check:")
python_version = !source /content/activate_env.sh && python --version
python_full_version = !source /content/activate_env.sh && python -c "import sys; print(sys.version)"

print(f"🐍 Python Version: {python_version[0].strip()}")
print(f"📋 Full Version Info: {python_full_version[0].strip()}")

# Verify PyTorch installation
print("\n🔍 PyTorch Check:")
torch_version = !source /content/activate_env.sh && python -c "import torch; print(f'PyTorch: {torch.__version__}')"
cuda_available = !source /content/activate_env.sh && python -c "import torch; print(f'CUDA Available: {torch.cuda.is_available()}')"

print(f"{torch_version[0].strip()}")
print(f"{cuda_available[0].strip()}")

if torch.cuda.is_available():
    cuda_version = !source /content/activate_env.sh && python -c "import torch; print(f'CUDA Version: {torch.version.cuda}')"
    gpu_name = !source /content/activate_env.sh && python -c "import torch; print(f'GPU: {torch.cuda.get_device_name(0)}')"
    gpu_memory = !source /content/activate_env.sh && python -c "import torch; print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB')"

    print(f"{cuda_version[0].strip()}")
    print(f"{gpu_name[0].strip()}")
    print(f"{gpu_memory[0].strip()}")

# Verify KRVC kernel availability
print("\n🔍 KRVC Kernel Check:")
try:
    !source /content/activate_env.sh && python -c "from advanced_rvc_inference.krvc_kernel import KRVCFeatureExtractor, krvc_speed_optimize; print('✅ KRVC Kernel: Available')"
    krvc_available = True
except Exception as e:
    print(f"❌ KRVC Kernel: Not available - {e}")
    krvc_available = False

# Verify Gradio version
print("\n🔍 Gradio Check:")
gradio_version = !source /content/activate_env.sh && python -c "import gradio; print(f'Gradio: {gradio.__version__}')"
print(f"{gradio_version[0].strip()}")

# Show available models and audio
print("\n🔍 Resource Check:")
model_dir = "/content/Advanced-RVC-Inference/logs/"
audio_dir = "/content/Advanced-RVC-Inference/audio_files/original_files"

if os.path.exists(model_dir):
    models = [d for d in os.listdir(model_dir) if os.path.isdir(os.path.join(model_dir, d))]
    print(f"📚 Available Models: {len(models)}")
    for model in models[:3]:
        print(f"   - {model}")
    if len(models) > 3:
        print(f"   ... and {len(models) - 3} more models")
else:
    print("📚 No models directory found")

if os.path.exists(audio_dir):
    audio_files = [f for f in os.listdir(audio_dir) if any(f.endswith(ext) for ext in ['.wav', '.mp3', '.flac', '.m4a'])]
    print(f"🎵 Available Audio Files: {len(audio_files)}")
    for audio in audio_files[:3]:
        print(f"   - {audio}")
    if len(audio_files) > 3:
        print(f"   ... and {len(audio_files) - 3} more audio files")
else:
    print("🎵 No audio files found")

# Display verification summary
status_color = "#4CAF50" if "Python 3.11" in python_version[0] and torch.cuda.is_available() else "#FFC107"
status_text = "✅ All systems go!" if "Python 3.11" in python_version[0] and torch.cuda.is_available() else "⚠️ Partial compatibility"

display(HTML(f"""
<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 15px; color: white; margin: 10px 0;'>
    <h3 style='margin: 0; text-align: center;'>{status_text}</h3>
    <div style='text-align: center; margin: 15px 0;'>
        <span style='background: rgba(255,255,255,0.2); padding: 8px 15px; border-radius: 20px; margin: 5px; display: inline-block;'>🐍 Python 3.11</span>
        <span style='background: rgba(255,255,255,0.2); padding: 8px 15px; border-radius: 20px; margin: 5px; display: inline-block;'>🔥 PyTorch {torch.__version__}</span>
        <span style='background: rgba(255,255,255,0.2); padding: 8px 15px; border-radius: 20px; margin: 5px; display: inline-block;'>{'' if not torch.cuda.is_available() else '🎮 GPU: ' + gpu_name[0].split(': ')[1]}</span>
        <span style='background: rgba(255,255,255,0.2); padding: 8px 15px; border-radius: 20px; margin: 5px; display: inline-block;'>{'' if not krvc_available else '⚡ KRVC Kernel'}</span>
    </div>
    <p style='text-align: center; margin: 10px 0;'>✅ Advanced RVC Inference is ready to use!</p>
    <div style='text-align: center; margin-top: 15px;'>
        <button onclick='alert("Environment verified successfully!")' style='background: #ffd700; color: #000; border: none; padding: 10px 20px; border-radius: 25px; font-weight: bold;'>🎉 Launch Interface</button>
    </div>
</div>
"""))

print("\n✅ Environment verification completed successfully!")
print("🚀 The Advanced RVC Inference - KRVC Kernel is ready for use with Python 3.11!")